
[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#!pip install web3
#!pip install dotenv
from web3 import Web3
import json
import os
from dotenv import load_dotenv

load_dotenv()

infura_url = os.getenv("INFURA_URL")
web3 = Web3(Web3.HTTPProvider(infura_url))

print("✅ Connected" if web3.is_connected() else "❌ Not connected")

contract_address = Web3.to_checksum_address(os.getenv("CONTRACT_ADDRESS"))
wallet_address = Web3.to_checksum_address(os.getenv("WALLET_ADDRESS"))
recipient = Web3.to_checksum_address(os.getenv("RECIPIENT"))
private_key = os.getenv("PRIVATE_KEY") 

with open("abi.json") as f:
    abi = json.load(f)

contract = web3.eth.contract(address=contract_address, abi=abi)

def simulate_sensor(sensor_id, recipient):
    print(f"📡 Sensor {sensor_id} uploading...")

    nonce = web3.eth.get_transaction_count(wallet_address)

    txn = contract.functions.reward(recipient, 1).build_transaction({
        'chainId': 11155111,
        'gas': 200000,
        'gasPrice': web3.to_wei('3', 'gwei'),
        'nonce': nonce
    })

    signed_txn = web3.eth.account.sign_transaction(txn, private_key=private_key)

    # ✅ FIX HERE: Use raw_transaction (all lowercase)
    tx_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)

    print("✅ Tx sent. Hash:", tx_hash.hex())
    tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash, timeout=300)
    print("⛏️ Mined in block:", tx_receipt.blockNumber)

    if tx_receipt.status == 1:
        print("✅ Transaction succeeded")
    else:
        print("❌ Transaction failed")

    

simulate_sensor("Sensor-A", "0x7f76b4015e08b24313c56Cc5e34489001144BDE7")


✅ Connected
📡 Sensor Sensor-A uploading...
✅ Tx sent. Hash: 3c6a611655f603c793edfb90123d3b7305f8c4c4b7e780799dcb7e2fef74828c
⛏️ Mined in block: 8867478
✅ Transaction succeeded


In [2]:

owner_balance = contract.functions.balanceOf(wallet_address).call()
recipient_balance = contract.functions.balanceOf(recipient).call()
print("📦 Mobile Account NRC balance:", web3.from_wei(owner_balance, 'ether'))
print("📥 Account 1 NRC balance:", web3.from_wei(recipient_balance, 'ether'))

📦 Mobile Account NRC balance: 999994
📥 Account 1 NRC balance: 6


In [3]:
contract.functions.decimals().call()


18